![](https://img-blog.csdnimg.cn/2021041710210731.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2JpdDQ1Mg==,size_16,color_FFFFFF,t_70)

![](https://img-blog.csdnimg.cn/20201114105033866.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2JpdDQ1Mg==,size_16,color_FFFFFF,t_70)

![](https://img-blog.csdnimg.cn/20201114105210740.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2JpdDQ1Mg==,size_16,color_FFFFFF,t_70)

![](https://img-blog.csdnimg.cn/20201114105215682.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2JpdDQ1Mg==,size_16,color_FFFFFF,t_70)

In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
 
 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
 
    def forward(self, x):
        x = x.view(-1, 784).cuda()  # -1其实就是自动获取mini_batch  
        x = F.relu(self.l1(x)).cuda()
        x = F.relu(self.l2(x)).cuda()
        x = F.relu(self.l3(x)).cuda()
        x = F.relu(self.l4(x)).cuda()
        x = self.l5(x).cuda()
        return x  # 最后一层不做激活，不进行非线性变换
 
 
model = Net()
model.cuda()
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
criterion.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        inputs = inputs.cuda()
        target = target.cuda()
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            # print("predicted = ",predicted,"\tlabel = ",labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.205
[1,   600] loss: 0.812
[1,   900] loss: 0.401
accuracy on test set: 89 % 
[2,   300] loss: 0.315
[2,   600] loss: 0.256
[2,   900] loss: 0.216
accuracy on test set: 94 % 
[3,   300] loss: 0.180
[3,   600] loss: 0.167
[3,   900] loss: 0.151
accuracy on test set: 95 % 
[4,   300] loss: 0.131
[4,   600] loss: 0.118
[4,   900] loss: 0.113
accuracy on test set: 96 % 
[5,   300] loss: 0.090
[5,   600] loss: 0.099
[5,   900] loss: 0.091
accuracy on test set: 96 % 
[6,   300] loss: 0.072
[6,   600] loss: 0.078
[6,   900] loss: 0.074
accuracy on test set: 97 % 
[7,   300] loss: 0.060
[7,   600] loss: 0.062
[7,   900] loss: 0.060
accuracy on test set: 97 % 
[8,   300] loss: 0.049
[8,   600] loss: 0.049
[8,   900] loss: 0.049
accuracy on test set: 97 % 
[9,   300] loss: 0.037
[9,   600] loss: 0.044
[9,   900] loss: 0.043
accuracy on test set: 97 % 
[10,   300] loss: 0.032
[10,   600] loss: 0.033
[10,   900] loss: 0.035
accuracy on test set: 97 % 
